In [1]:
!git clone https://github.com/abdelrahman-esmat/python-notebooks

Cloning into 'python-notebooks'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 15 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (15/15), done.


### Step 0
Load the energy data from the file `Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']`

Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, 

e.g. 

`'Bolivia (Plurinational State of)'` should be `'Bolivia'`, 

`'Switzerland17'` should be `'Switzerland'`.

In [2]:
import numpy as np
import pandas as pd

In [3]:
#load data set
energy=pd.read_excel("/content/python-notebooks/data preprocessing using pandas/data/Energy Indicators.xls",skiprows=16,usecols="C:F",nrows=228,names=['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable'])

In [4]:
#dropping second row which is useless
energy.drop(index=0,axis=0,inplace=True)
energy.reset_index(drop=True,inplace=True)

In [5]:
#convert from petajoule to gigajoule
energy['Energy Supply']=energy['Energy Supply']*1000000

In [6]:
#replace non numeric values with nan
energy['Energy Supply'] = pd.to_numeric(energy['Energy Supply'], errors='coerce')


In [7]:
#fix some data entry errors and remove unwanted brackets
energy['Country']=energy['Country'].replace(to_replace='([0-9])|(\s*\(([a-zA-Z]*\s*)*\))',regex=True,value='')

In [8]:
#change name of countries
energy=energy.replace({"Country":{"Republic of Korea": "South Korea", "United States of America": "United States", "United Kingdom of Great Britain and Northern Ireland": "United Kingdom", "China, Hong Kong Special Administrative Region": "Hong Kong"}})

### Step 1
<br>

Next, load the GDP data from the file `world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

<br>

In [9]:
gdp=pd.read_csv("/content/python-notebooks/data preprocessing using pandas/data/world_bank.csv",header=4)

In [10]:
gdp['Country Name'].replace({"Korea, Rep.": "South Korea", "Iran, Islamic Rep.": "Iran", "Hong Kong SAR, China": "Hong Kong"},inplace=True)

In [11]:
gdp.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,Aruba,ABW,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.467704e+09,NaN,NaN,NaN,NaN,NaN
1,Andorra,AND,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.022871e+09,1.070430e+09,1.157667e+09,1.247831e+09,1.317944e+09,1.325090e+09,1.368868e+09,1.407725e+09,1.428320e+09,1.428913e+09,1.460474e+09,1.458539e+09,1.476720e+09,1.502859e+09,1.529681e+09,1.565191e+09,1.616112e+09,1.705759e+09,1.792656e+09,1.879188e+09,1.950248e+09,1.999901e+09,2.018485e+09,1.997664e+09,2.045272e+09,2.101671e+09,2.199393e+09,2.398827e+09,2.475464e+09,2.576935e+09,2.606977e+09,2.765137e+09,2.942848e+09,3.296693e+09,3.562720e+09,3.842027e+09,4.018196e+09,4.021331e+09,3.675728e+09,3.535389e+09,3.346317e+09,3.185605e+09,3.129538e+09,3.127550e+09,NaN,NaN
2,Afghanistan,AFG,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.013233e+09,8.689884e+09,8.781610e+09,9.762979e+09,1.030523e+10,1.172119e+10,1.214448e+10,1.469733e+10,1.593680e+10,1.691113e+10,1.935220e+10,1.973134e+10,1.999032e+10,2.029415e+10
3,Angola,AGO,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.730822e+10,2.807285e+10,3.029060e+10,3.198688e+10,3.211482e+10,3.201848e+10,3.163426e+10,2.945149e+10,2.217698e+10,2.295317e+10,2.534030e+10,2.817841e+10,3.040451e+10,3.247346e+10,3.352554e+10,3.453535e+10,3.599308e+10,3.349499e+10,3.525264e+10,3.908798e+10,4.622601e+10,5.581103e+10,6.842044e+10,7.787420e+10,7.975320e+10,8.247091e+10,8.570262e+10,9.012096e+10,9.626143e+10,1.008863e+11,1.039106e+11
4,Albania,ALB,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.442066e+09,5.754748e+09,5.924432e+09,5.989893e+09,5.914924e+09,6.020248e+09,6.359624e+09,6.309520e+09,6.219922e+09,6.831748e+09,6.177564e+09,4.349685e+09,4.036508e+09,4.424012e+09,4.791205e+09,5.428436e+09,5.922423e+09,5.280560e+09,5.756278e+09,6.533443e+09,6.969002e+09,7.522364e+09,7.840663e+09,8.293343e+09,8.766856e+09,9.268393e+09,9.771760e+09,1.034829e+10,1.112752e+10,1.150029e+10,1.192695e+10,1.223109e+10,1.240477e+10,1.254247e+10,1.279331e+10,1.312082e+10


### Step 2
Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

In [12]:
ScimEn=pd.read_excel('/content/python-notebooks/data preprocessing using pandas/data/scimagojr-3.xlsx')

In [13]:
ScimEn.head()

,Rank,Country,Documents,Citable documents,Citations,Self-citations,Citations per document,H index
0,1,China,127050,126767,597237,411683,4.70,138
1,2,United States,96661,94747,792274,265436,8.20,230
2,3,Japan,30504,30287,223024,61554,7.31,134
3,4,United Kingdom,20944,20357,206091,37874,9.84,139
4,5,Russian Federation,18534,18301,34266,12422,1.85,57


### Step 3
Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This step should yeild a DataFrame with 20 columns and 15 entries.*

In [14]:
df1=energy.set_index('Country')

In [15]:
select=list(range(-10,0))+[0]
df2=gdp.iloc[:,select].set_index('Country Name')

In [16]:
df3=ScimEn.iloc[:15,:].set_index('Country')

In [17]:
all=df3.join([df1,df2],how='inner')

### Step 4
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This step should yield a single number.*

In [35]:
df1=energy.set_index('Country')

In [36]:
select=list(range(-10,0))+[0]
df2=gdp.iloc[:,select].set_index('Country Name')

In [37]:
df3=ScimEn.iloc[:,:].set_index('Country')

In [38]:
tmp=df3.join([df1,df2],how='inner')

In [40]:
print(tmp.shape[0]-all.shape[0])

147


### Step 5

#### Answer the following questions in the context of only the top 15 countries by Scimagojr Rank 


What is the average GDP over the last 10 years for each country? (exclude missing values from this calculation.)

*This step should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [18]:
all.iloc[:,-10:].mean(axis=1)

China                 6.348609e+12
United States         1.536434e+13
Japan                 5.542208e+12
United Kingdom        2.487907e+12
Russian Federation    1.565459e+12
Canada                1.660647e+12
Germany               3.493025e+12
India                 1.769297e+12
France                2.681725e+12
South Korea           1.106715e+12
Italy                 2.120175e+12
Spain                 1.418078e+12
Iran                  4.441558e+11
Australia             1.164043e+12
Brazil                2.189794e+12
dtype: float64

### Step  6
What is the mean `Energy Supply per Capita`?

*This step should return a single number.*

In [19]:
all['Energy Supply per Capita'].mean()

157.6

### Step 7
What country has the maximum % Renewable and what is the percentage?

*This step should return a tuple with the name of the country and the percentage.*

In [20]:
all

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
China,1,127050,126767,597237,411683,4.70,138,1.271910e+11,93,19.7549,3.992331e+12,4.559041e+12,4.997775e+12,5.459247e+12,6.039659e+12,6.612490e+12,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12
United States,2,96661,94747,792274,265436,8.20,230,9.083800e+10,286,11.571,1.479230e+13,1.505540e+13,1.501149e+13,1.459484e+13,1.496437e+13,1.520402e+13,1.554216e+13,1.577367e+13,1.615662e+13,1.654857e+13
Japan,3,30504,30287,223024,61554,7.31,134,1.898400e+10,149,10.2328,5.496542e+12,5.617036e+12,5.558527e+12,5.251308e+12,5.498718e+12,5.473738e+12,5.569102e+12,5.644659e+12,5.642884e+12,5.669563e+12
United Kingdom,4,20944,20357,206091,37874,9.84,139,7.920000e+09,124,10.6005,2.419631e+12,2.482203e+12,2.470614e+12,2.367048e+12,2.403504e+12,2.450911e+12,2.479809e+12,2.533370e+12,2.605643e+12,2.666333e+12
Russian Federation,5,18534,18301,34266,12422,1.85,57,3.070900e+10,214,17.2887,1.385793e+12,1.504071e+12,1.583004e+12,1.459199e+12,1.524917e+12,1.589943e+12,1.645876e+12,1.666934e+12,1.678709e+12,1.616149e+12
Canada,6,17899,17620,215003,40930,12.01,149,1.043100e+10,296,61.9454,1.564469e+12,1.596740e+12,1.612713e+12,1.565145e+12,1.613406e+12,1.664087e+12,1.693133e+12,1.730688e+12,1.773486e+12,1.792609e+12
Germany,7,17027,16831,140566,27426,8.26,126,1.326100e+10,165,17.9015,3.332891e+12,3.441561e+12,3.478809e+12,3.283340e+12,3.417298e+12,3.542371e+12,3.556724e+12,3.567317e+12,3.624386e+12,3.685556e+12
India,8,15005,14841,128763,37209,8.58,115,3.319500e+10,26,14.9691,1.265894e+12,1.374865e+12,1.428361e+12,1.549483e+12,1.708459e+12,1.821872e+12,1.924235e+12,2.051982e+12,2.200617e+12,2.367206e+12
France,9,13153,12973,130632,28601,9.93,114,1.059700e+10,166,17.0203,2.607840e+12,2.669424e+12,2.674637e+12,2.595967e+12,2.646995e+12,2.702032e+12,2.706968e+12,2.722567e+12,2.729632e+12,2.761185e+12
South Korea,10,11983,11923,114675,22595,9.57,104,1.100700e+10,221,2.27935,9.410199e+11,9.924316e+11,1.020510e+12,1.027730e+12,1.094499e+12,1.134796e+12,1.160809e+12,1.194429e+12,1.234340e+12,1.266580e+12


In [21]:
print("the maximum percentage is ", all['% Renewable'].max(), " and the country is : ",all.loc[all['% Renewable' ] == all['% Renewable'].max()].index[0])

the maximum percentage is  69.64803  and the country is :  Brazil


### Step 8
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This step should return a tuple with the name of the country and the ratio.*

In [22]:
all['ratio']=all['Self-citations']/all['Citations']

In [23]:
(all.loc[all['ratio' ] == all['ratio'].max()].index[0],all['ratio'].max())

('China', 0.6893126179389422)

### Step 9
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

*This step should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [24]:
mm=all['% Renewable'].median()
all['HighRenew']=1

In [32]:
all.loc[all['% Renewable']<mm,'HighRenew']=0

In [34]:
all.sort_values('Rank')['HighRenew']

China                 1
United States         0
Japan                 0
United Kingdom        0
Russian Federation    1
Canada                1
Germany               1
India                 0
France                1
South Korea           0
Italy                 1
Spain                 1
Iran                  0
Australia             0
Brazil                1
Name: HighRenew, dtype: int64

### Step 10
Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [41]:
all['continent']=all.index

In [44]:
all.replace({'continent': {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}},inplace=True)

In [56]:
all['est_pop']=(all['Energy Supply']/all['Energy Supply per Capita']).astype(int)

In [73]:
output=all.groupby('continent')['est_pop'].describe().iloc[:,:3]

In [75]:
output.replace(to_replace=np.NaN,value=0,inplace=True)

In [76]:
output['sum']=all.groupby('continent')['est_pop'].sum()

In [77]:
output

,count,mean,std,sum
continent,,,,
Asia,5.0,5.797333e+08,6.790979e+08,2898666384
Australia,1.0,2.331602e+07,0.000000e+00,23316017
Europe,6.0,7.632161e+07,3.464767e+07,457929664
North America,2.0,1.764276e+08,1.996696e+08,352855248
South America,1.0,2.059153e+08,0.000000e+00,205915254
